In [25]:
from pathlib import Path
import mne
from mne_bids import BIDSPath, get_entity_vals, read_raw_bids, get_entities_from_fname
import numpy as np
from scipy.integrate import simps
from mne.time_frequency import psd_array_multitaper
import json

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def bandpower(data, sf, band, ch_names):
    """
    Calculate power in the given frequency band for the provided channels.

    Parameters
    ----------
    data: np.ndarray
        nch x nSamp matrix containing the eeg data
    sf: int
        Sampling frequency
    band: list[int]
        List in the form of [lFreq, hFreq]
    ch_names: list[str]
        List of channel names

    Returns
    -------
    band_dict: dict
        Dictionary where keys are channel names and values are power in the provided band.

    """
    psd, freqs = psd_array_multitaper(data, sf, band[0], band[1], adaptive=True, normalization='full', verbose=0)
    # Calculate frequency resolution
    freq_res = freqs[1] - freqs[0]
    # Specify band range
    idx_band = np.logical_and(freqs >= band[0], freqs <= band[1])
    # PSD is a series of discrete values, cannot be directly integrated, so use SIMPS to make a parabolic approximation
    band_power = simps(psd[:,idx_band], dx=freq_res)
    band_dict = {}
    for ind, bp in enumerate(band_power):
        band_dict[ch_names[ind]] = bp
    return band_dict


In [17]:
bids_root = "D:/OneDriveParent/OneDrive - Johns Hopkins/Shared Documents/40Hz-30"
sourcedata = Path(bids_root) / "sourcedata"
deriv_dir = Path(bids_root) / "derivatives"

In [21]:
session = 'initialvisit'
task = 'monitor'
run = "01"
datatype = "eeg"
montage = "standard_1020"
ext = ".vhdr"
reference="monopolar"

In [22]:
ignore_subjects = None  #['00002744', '3']
subjects = get_entity_vals(bids_root, 'subject', ignore_subjects=ignore_subjects)
bids_fpaths = [BIDSPath(subject=s, session=session, task=task, run=run, datatype=datatype, extension=ext, root=bids_root).match() for s in subjects]
bids_fpaths = [item for sublist in bids_fpaths for item in sublist]

In [7]:
keep_chs = ['Fp1', 'Fp2', 'F3','F4', 'F7', 'F8', 'Fz','T3', 'T4', 'C3', 'C4', 'Cz', 'T5', 'T6', 'P3', 'P4', 'Pz', 'O1', 'O2', 'T7', 'T8', 'P7', 'P8']

In [8]:
def get_channel_map(raw_chs, rename_chs):
    ch_map = {}
    for ch in rename_chs:
        for rch in raw_chs:
            rch_base = rch.split("-")[0]
            if ch.upper() == rch_base.upper():
                ch_map[rch] = ch
    return ch_map

In [9]:
bands = [[1, 3], [4, 7], [8, 12], [13, 30]]
band_names = ['delta', 'theta', 'alpha', 'beta']

In [30]:
for fpath in bids_fpaths:
    raw_bids_entities = get_entities_from_fname(fpath)
    subject = raw_bids_entities['subject']
    fname = Path(str(fpath)).name
    
    raw = read_raw_bids(fpath)
    raw.pick_types(eeg=True)
    rename_map = get_channel_map(raw.ch_names, keep_chs)
    raw.rename_channels(get_channel_map(raw.ch_names, keep_chs))
    raw.pick_channels(keep_chs)
    data = raw.get_data()
    sfreq = raw.info['sfreq']
    ch_names = raw.ch_names
    
    band_root = deriv_dir / "band-power" / "monopolar"
    band_dir = band_root / f"sub-{subject}"
    band_fname = f"{fname.split('.')[0]}_bandpower.json"
    band_fpath = band_dir / band_fname
    Path(band_fpath.parent).mkdir(parents=True, exist_ok=True)
    
    
    pows = {}
    for band, band_name in zip(bands, band_names):
        pows[band_name] = bandpower(data, sfreq, band, ch_names)
    with open(band_fpath, 'w+') as f:
        json.dump(pows, f, indent=4)
    

Extracting parameters from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-001\ses-initialvisit\eeg\sub-001_ses-initialvisit_task-monitor_run-01_eeg.vhdr...
Setting channel info structure...
Reading channel info from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-001\ses-initialvisit\eeg\sub-001_ses-initialvisit_task-monitor_run-01_channels.tsv.
Extracting parameters from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-002\ses-initialvisit\eeg\sub-002_ses-initialvisit_task-monitor_run-01_eeg.vhdr...
Setting channel info structure...
Reading channel info from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-002\ses-initialvisit\eeg\sub-002_ses-initialvisit_task-monitor_run-01_channels.tsv.
Extracting parameters from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-003\ses-initialvisit\eeg\sub-003_ses-initialvisit_task-monitor_run-01_eeg.vhdr...
Setting channel info stru

Extracting parameters from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-022\ses-initialvisit\eeg\sub-022_ses-initialvisit_task-monitor_run-01_eeg.vhdr...
Setting channel info structure...
Reading channel info from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-022\ses-initialvisit\eeg\sub-022_ses-initialvisit_task-monitor_run-01_channels.tsv.
Extracting parameters from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-023\ses-initialvisit\eeg\sub-023_ses-initialvisit_task-monitor_run-01_eeg.vhdr...
Setting channel info structure...
Reading channel info from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-023\ses-initialvisit\eeg\sub-023_ses-initialvisit_task-monitor_run-01_channels.tsv.
Extracting parameters from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-024\ses-initialvisit\eeg\sub-024_ses-initialvisit_task-monitor_run-01_eeg.vhdr...
Setting channel info stru

Extracting parameters from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-117\ses-initialvisit\eeg\sub-117_ses-initialvisit_task-monitor_run-01_eeg.vhdr...
Setting channel info structure...
Reading channel info from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-117\ses-initialvisit\eeg\sub-117_ses-initialvisit_task-monitor_run-01_channels.tsv.
Extracting parameters from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-118\ses-initialvisit\eeg\sub-118_ses-initialvisit_task-monitor_run-01_eeg.vhdr...
Setting channel info structure...
Reading channel info from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-118\ses-initialvisit\eeg\sub-118_ses-initialvisit_task-monitor_run-01_channels.tsv.
Extracting parameters from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-119\ses-initialvisit\eeg\sub-119_ses-initialvisit_task-monitor_run-01_eeg.vhdr...
Setting channel info stru

Extracting parameters from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-216\ses-initialvisit\eeg\sub-216_ses-initialvisit_task-monitor_run-01_eeg.vhdr...
Setting channel info structure...
Reading channel info from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-216\ses-initialvisit\eeg\sub-216_ses-initialvisit_task-monitor_run-01_channels.tsv.
Extracting parameters from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-217\ses-initialvisit\eeg\sub-217_ses-initialvisit_task-monitor_run-01_eeg.vhdr...
Setting channel info structure...
Reading channel info from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-217\ses-initialvisit\eeg\sub-217_ses-initialvisit_task-monitor_run-01_channels.tsv.
Extracting parameters from D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\40Hz-30\sub-218\ses-initialvisit\eeg\sub-218_ses-initialvisit_task-monitor_run-01_eeg.vhdr...
Setting channel info stru